<a href="https://colab.research.google.com/github/bogdanbabych/experiments_NLTK/blob/main/Intelligence_Task_Ontology_(ITO)_exploration_and_visualization_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install Owlready2
from owlready2 import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 32.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Owlready2: filename=Owlready2-0.41-cp310-cp310-linux_x86_64.whl size=24170136 sha256=29037be0d9878456e63a23ec40c7f6286c9d3c8d3a3e60934dfb6deb356c2c08
  Stored in directory: /root/.cache/pip/wheels/da/02/c0/a65e08daab6da8123998823c9aee7bba656b1500ab05d183ef
Successfully built Owlready2


In [5]:
!wget http://aber-owl.net/media/ontologies/ITO/17/ito.owl

--2023-05-11 08:31:18--  http://aber-owl.net/media/ontologies/ITO/17/ito.owl
Resolving aber-owl.net (aber-owl.net)... 45.223.20.138
Connecting to aber-owl.net (aber-owl.net)|45.223.20.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53946229 (51M) [application/octet-stream]
Saving to: ‘ito.owl’

ito.owl             100%[===================>]  51.45M  3.58MB/s    in 16s     

2023-05-11 08:31:34 (3.24 MB/s) - ‘ito.owl’ saved [53946229/53946229]



In [7]:
onto = get_ontology("file:///content/ito.owl")
# http://aber-owl.net/media/ontologies/ITO/17/ito.owl
# onto = get_ontology("https://github.com/OpenBioLink/ITO/raw/master/ITO.owl")
# onto = get_ontology("http://aber-owl.net/media/ontologies/ITO/17/ito.owl")

onto.load()
ITO = onto.get_namespace("https://ai-strategies.org/ontology/")
EDAM = onto.get_namespace("http://edamontology.org/")

# Creating ontology statistics

Please note that the queries in this example Colab notebook are run in the notebook with a slow Python RDF store, and more complex queries can have a long runtime. Use a performant SPARQL store such as https://blazegraph.com/ for fast queries (which can be queried with Python through the https://github.com/RDFLib/sparqlwrapper library).

In [8]:
! pip install rdflib
import rdflib
graph = default_world.as_rdflib_graph()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.2 MB/s eta 0:00:00


In [9]:
import pandas as pd
import json
def sparql_result_to_df(sparql_result):
    # converts SPARQL SELECT results to Pandas dataframe
    
    cols = sparql_result.bindings[0].keys()

    out = []
    for row in sparql_result.bindings:
        item = []
        for c in cols:
            item.append(row.get(c, {}))
        out.append(item)
    return pd.DataFrame(out, columns=cols)

In [10]:
namespaces ={ 'owl': rdflib.OWL,
              'rdfs': rdflib.RDFS,
              'edam': 'http://edamontology.org/',
              'ito': 'https://ai-strategies.org/ontology/'}

### Are there entities/properties with more than one label? (data quality check)

In [11]:
result = graph.query("""
SELECT ?entity 
       (COUNT(?entity_label) AS ?entity_label_count)
WHERE {
    ?entity rdfs:label ?entity_label .
}
GROUP BY ?entity
ORDER BY DESC(?entity_label_count)

""", initNs = namespaces)
df = sparql_result_to_df(result)
df

,entity,entity_label_count
0,https://identifiers.org/ito:ontology,2
1,http://edamontology.org/citation,1
2,http://edamontology.org/data_0006,1
3,http://edamontology.org/data_0842,1
4,http://edamontology.org/documentation,1
...,...,...
56697,https://identifiers.org/ito:ITO_63088,1
56698,https://identifiers.org/ito:ITO_63089,1
56699,https://identifiers.org/ito:ITO_63090,1
56700,https://identifiers.org/ito:ITO_63091,1


### Property stats on individual level (how many measurements with each property?)

In [12]:
result = graph.query("""
SELECT ?performance_measure_label ?performance_measure
       (COUNT(?benchmark_process_indivdual) AS ?benchmark_process_indivdual_count)
WHERE {
    ?performance_measure rdfs:subPropertyOf* ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    ?benchmark_process_indivdual ?performance_measure ?result .
}
GROUP BY ?performance_measure
ORDER BY DESC(?benchmark_process_indivdual_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('direct_property_statistics.csv', index=False)
df

""
0


In [13]:
result = graph.query("""
SELECT ?performance_measure_label ?performance_measure
       (COUNT(?benchmark_process_indivdual_in_subproperty) AS ?benchmark_process_individual_in_subproperty_count)
WHERE {
    ?performance_measure rdfs:subPropertyOf ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    ?performance_measure_subproperty rdfs:subPropertyOf* ?performance_measure .
    ?benchmark_process_indivdual_in_subproperty ?performance_measure_subproperty ?result_s .
}

GROUP BY ?performance_measure
ORDER BY DESC(?benchmark_process_individual_in_subproperty_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('subproperty_statistics.csv', index=False)
df

""
0


### Property stats on benchmark level (how many benchmarks with each property?)

In [14]:
result = graph.query("""
SELECT ?performance_measure_label ?performance_measure
       (COUNT(?benchmark) AS ?benchmark_count)
WHERE {
    ?performance_measure rdfs:subPropertyOf* ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    ?benchmark rdfs:subClassOf ?property_restriction .
    ?property_restriction owl:onProperty ?performance_measure
}
GROUP BY ?performance_measure
ORDER BY DESC(?benchmark_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('direct_property_statistics_per_benchmark.csv', index=False)
df

""
0


## Total number of benchmarks

In [15]:
result = graph.query("""
SELECT (COUNT(?benchmark) as ?benchmark_count) WHERE {
    ?benchmark rdfs:subClassOf ito:Benchmarking
}
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('total_number_of_benchmarks.csv', index=False)
df

,benchmark_count
0,0


## Number of different metrics per benchmark (for those where any metrics are listed) 



In [16]:
result = graph.query("""
SELECT ?benchmark_label ?benchmark
       (COUNT(DISTINCT?performance_measure) AS ?performance_measure_count)
       (GROUP_CONCAT(DISTINCT ?performance_measure_label;SEPARATOR=";") AS ?performance_measure_labels)
WHERE {
    ?performance_measure rdfs:subPropertyOf* ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    ?benchmark rdfs:subClassOf ito:Benchmarking ;
              rdfs:label ?benchmark_label ;
              rdfs:subClassOf ?property_restriction .
    ?property_restriction owl:onProperty ?performance_measure .
}
GROUP BY ?benchmark_label ?benchmark
ORDER BY DESC(?performance_measure_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('number-of-different-metrics-per-benchmark.csv', index=False)
df

""
0


## Number of different metrics per benchmark (for those where any metrics are listed -- with topmost properties)


In [17]:
result = graph.query("""
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix ito: <https://ai-strategies.org/ontology/>
SELECT  ?benchmark_label ?benchmark
       (COUNT(DISTINCT ?performance_measure) AS ?performance_measure_count)
       (GROUP_CONCAT(DISTINCT ?performance_measure_label;SEPARATOR=";") AS ?performance_measure_labels)
WHERE {
    {
        ?performance_measure rdfs:subPropertyOf ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    }
    UNION
    {
        ?performance_measure rdfs:subPropertyOf* ?performance_measure_topmost .
        ?performance_measure_topmost rdfs:subPropertyOf ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    }
    ?benchmark rdfs:subClassOf ito:Benchmarking ;
              rdfs:label ?benchmark_label ;
              rdfs:subClassOf ?property_restriction .
    ?property_restriction owl:onProperty ?performance_measure .
}
GROUP BY ?benchmark_label ?benchmark
ORDER BY DESC(?performance_measure_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('number-of-different-metrics-per-benchmark-superproperties.csv', index=False)
df

""
0


## Total number of papers

In [18]:
result = graph.query("""
SELECT (COUNT(?paper) as ?paper_count) WHERE {
    ?paper a edam:data_0971 .
}
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('total_number_of_papers.csv', index=False)
df

,paper_count
0,11749


## Number of papers that have at least one annotated metric

In [19]:
result = graph.query("""
SELECT (COUNT(DISTINCT ?paper) as ?paper_count) WHERE {
    ?paper a edam:data_0971 . 
    ?benchmark_process_individual rdfs:seeAlso ?paper . 
    ?benchmark_process_individual ?performance_measure ?result .
    ?performance_measure rdfs:subPropertyOf* ito:performance_measure .
}
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('number-of-papers-that-have-at-least-one-annotated-metric.csv', index=False)
df

,paper_count
0,0


## Time span of publications covered by PWC

In [20]:
result = graph.query("""
SELECT ?paper ?date WHERE {
    ?paper a edam:data_0971 .
    ?paper <http://purl.obolibrary.org/obo/date> ?date
}
ORDER BY ?date
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('time_span_of_publications_covered_by_PWC.csv', index=False)
df

IndexError: ignored

## Number of metrics aggregated by high-level process / task 

In [21]:
result = graph.query("""
SELECT ?high_level_process_label ?high_level_process
       (COUNT(?subprocess_performance_measure) AS ?subprocess_performance_measure_count)
WHERE {
    ?high_level_process rdfs:subClassOf ito:Process ;
                        rdfs:label ?high_level_process_label . 

    ?subprocess rdfs:subClassOf* ?high_level_process .

    ?subprocess rdfs:subClassOf ?property_restriction .
    ?property_restriction owl:onProperty ?subprocess_performance_measure .
    ?subprocess_performance_measure rdfs:subPropertyOf* ito:performance_measure .
}
GROUP BY ?high_level_process
ORDER BY DESC(?subprocess_performance_measure_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('number_of_metrics_aggregated_by_high_level_process.csv', index=False)
df

""
0


## Number of benchmarks for each high-level process


In [22]:
result = graph.query("""
SELECT ?high_level_process_label ?high_level_process
       (COUNT(?benchmark) AS ?benchmark_count)
WHERE {
    ?high_level_process rdfs:subClassOf ito:Process ;
                        rdfs:label ?high_level_process_label . 

    ?benchmark rdfs:subClassOf ito:Benchmarking . 
    ?benchmark rdfs:subClassOf* ?high_level_process .
}
GROUP BY ?high_level_process
ORDER BY DESC(?benchmark_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('number_of_benchmarks_per_high_level_process.csv', index=False)
df

""
0


In [ ]:
! zip results.zip *.csv 

## Code for generating JSON for hierarchy visualization in d3js

In [23]:
def class_to_dict(myclass):
    if myclass == ITO.Benchmarking:
        mydict = dict()
        mydict['name'] = "Benchmarking"
        mydict['size'] = 1
        return mydict
    mydict = dict()
    try:
        mydict['name'] = myclass.label[0]
    except IndexError:
        mydict['name'] = 'Class'
    children = list()
    for subclass in myclass.subclasses():
        children.append(class_to_dict(subclass))
    if len(children) > 0:
        mydict['children'] = children
    mydict['value'] = 1
    return mydict

In [24]:
mydict = class_to_dict(ITO.Process)

In [25]:
import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(mydict, f, indent=2)

In [26]:
result = graph.query("""
SELECT ?performance_measure_label ?performance_measure
       (COUNT(?benchmark) AS ?benchmark_count)
WHERE {
    ?performance_measure rdfs:subPropertyOf ito:performance_measure ;
                         rdfs:label ?performance_measure_label .
    ?performance_measure_subproperty rdfs:subPropertyOf* ?performance_measure .
    ?benchmark rdfs:subClassOf ?property_restriction .
    ?property_restriction owl:onProperty ?performance_measure_subproperty
}
GROUP BY ?performance_measure
ORDER BY DESC(?benchmark_count)
""", initNs = namespaces)
df = sparql_result_to_df(result)
df.to_csv('suproperty_statistics_per_benchmark.csv', index=False)
df

""
0
